<a href="https://colab.research.google.com/github/michel-vbarboza/ERA5-Temperature-Prediction/blob/main/LSTM_Neural_Network_(GPU).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#!pip install optuna -q

In [ ]:
import pickle
import pandas as pd
import numpy as np
#import optuna
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/ERA5-Temperature-Prediction/Dados/'

# Preprocessing

In [ ]:
with open(path+'train.pkl', 'rb') as f:
    train = pickle.load(f)

with open(path+'test.pkl', 'rb') as f:
    test = pickle.load(f)

del f

In [ ]:
targets = ['target_1', 'target_3', 'target_5',
           'target_7', 'target_10', 'target_15']

features = train.columns.difference(targets)

In [ ]:
target = 'target_10'

In [ ]:
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dropout, Dense, Input, LayerNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from tensorflow.keras.mixed_precision import set_global_policy

# Enable GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Enable XLA (Accelerated Linear Algebra) compilation
tf.config.optimizer.set_jit(True)

# Set up mixed precision training
set_global_policy('mixed_float16')

# Verify GPU usage and mixed precision
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Mixed precision policy:", tf.keras.mixed_precision.global_policy())

# Define the enhanced LSTM model
model = Sequential()

# Input layer specifying the input shape (timesteps, num_features)
model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.3))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(1, dtype='float32'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), verbose=1)

# Predict on test data
y_pred = model.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error on Test Data: {rmse}")


TensorFlow version: 2.17.0
Num GPUs Available:  1
Mixed precision policy: <DTypePolicy "mixed_float16">
Epoch 1/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 84s 6ms/step - loss: 37699.3984 - val_loss: 54.7119
Epoch 2/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 76s 6ms/step - loss: 719.4475 - val_loss: 12.3298
Epoch 3/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 86s 6ms/step - loss: 664.9692 - val_loss: 11.3396
Epoch 4/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 75s 6ms/step - loss: 624.9687 - val_loss: 11.1839
Epoch 5/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 81s 5ms/step - loss: 588.3242 - val_loss: 8.9564
Epoch 6/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 84s 6ms/step - loss: 555.3530 - val_loss: 11.1210
Epoch 7/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 85s 6ms/step - loss: 522.1101 - val_loss: 8.1234
Epoch 8/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 73s 5ms/step - loss: 488.8435 - val_loss: 9.3920
Epoch 9/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 75s 6ms/step - loss: 458.0981 - val_loss: 7.8677
Epoch 10/100
13355/13355 ━━━━━━━━━━━━━━━━━━

In [ ]:
# Save lists using pickle
with open(path+'pred_lstm_'+target+'.pkl', 'wb') as f:
    pickle.dump(y_pred, f)

with open(path+'rmse_lstm_'+target+'.pkl', 'wb') as f:
    pickle.dump(rmse, f)
